In [29]:
import os
import nltk
import torch
import string
from peft import PeftModel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [27]:
from nltk.stem import WordNetLemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [13]:

# Define paths
base_model_path = "./qwen"  # Base model path
lora_model_path = "Qwen2-1.5B-GRPO"  # LoRA fine-tuned model path

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_path)
tokenizer.pad_token = tokenizer.eos_token

# Load the base model with quantization (if needed)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(base_model_path, quantization_config=bnb_config, device_map="auto")
# Load the LoRA fine-tuned adapter
model = PeftModel.from_pretrained(base_model, lora_model_path)

# Merge LoRA weights for inference
model = model.merge_and_unload()


In [14]:
def summarize_text(text, max_length=150, min_length=50):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    summary_ids = model.generate(
        inputs.input_ids, max_length=max_length, min_length=min_length, length_penalty=2.0, early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [15]:
def get_text_files(directory):
    return [f for f in os.listdir(directory) if f.endswith('.txt')]

In [16]:
def read_files(file_list, directory):
    content = {}
    for filename in file_list:
        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
            content[filename] = file.read()
    return content

In [49]:
jdirectory = 'D://Legal//IND//IN-Abs//train-data//judgement'
text_files = get_text_files(jdirectory)
judgement_content = read_files(text_files, jdirectory)
sdirectory = 'D://Legal//IND//IN-Abs//train-data//summary'
text_files = get_text_files(sdirectory)
summary_content = read_files(text_files, sdirectory)
X = [text for fn, text in judgement_content.items()]
y = [text for fn, text in summary_content.items()]

In [19]:
def preprocess_text(text):
    text= text.lower()
    text= text.translate(str.maketrans('', '', string.punctuation))
    tokens= word_tokenize(text)
    stop_words= set(stopwords.words('english'))
    tokens = [word for word in tokens if not word in stop_words]
    lemmatizer= WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    nltk_words = set(nltk.corpus.words.words())
    corrected_tokens = []
    i = 0
    n = len(tokens)
    while i < n - 1:
        combined_word = tokens[i] + tokens[i + 1]
        if combined_word in nltk_words:
            corrected_tokens.append(combined_word)
            i += 2
        else:
            corrected_tokens.append(tokens[i])
            i += 1
    if i == n - 1:
        corrected_tokens.append(tokens[-1])
    return corrected_tokens

In [31]:
import itertools

for filename, content in itertools.islice(judgement_content.items(), 5):
    content = preprocess_text(content)

In [32]:
def generate_summary(text, model, tokenizer, max_length=128):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(model.device)

    with torch.no_grad():
        output = model.generate(
            input_ids=inputs["input_ids"],
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.7,
            top_k=50,
            top_p=0.95
        )

    return tokenizer.decode(output[0], skip_special_tokens=True)

In [35]:
for filename, content in itertools.islice(judgement_content.items(), 5):
    summary = generate_summary(content, model, tokenizer)
    print("Generated Summary:", summary)

C:\Users\vysha\ADA\Lib\site-packages\transformers\generation\configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\vysha\ADA\Lib\site-packages\transformers\generation\configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected be

Generated Summary: Appeal No. LXVI of 1949.
Appeal from the High Court of judicature, Bombay, in a reference under section 66 of the Indian Income tax Act, 1022.
K.M. Munshi (N. P. Nathvani, with him), for the appel lant. ' M.C. Setalvad, Attorney General for India (H. J. Umrigar, with him), for the respondent. 1950.
May 26.
The judgment of the Court was delivered by MEHR CHAND MAHAJAN J.
This is an appeal against a judgment of the High Court of Judicature at Bombay in an income tax matter and it raises the question whether munici pal property tax and urban immoveable property tax payable under the relevant Bombay Acts are allowable deductions under section 9 (1) (iv) of the Indian Income tax Act.
The assessee company is an investment company deriving its income from properties in the city of Bombay.
For the assessment year 1940 41 the net income of the assessee under the head "property" was computed by the Income tax Officer in the sum of Rs. 6,21,764 after deducting from gross rents 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated Summary: Civil Appeal No.94 of 1949.
107 834 Appeal from a judgment and decree of the High Court of Judi cature at Patna in Appeal from Appellate Decree No. 97 of 1946 (Mannohar Lall and Mukherji JJ.) dated 23rd Decem ber, 1947, confirming the judgment of the District Judge of Purulia in Appeal No. 159 of 1944.
S.P. Sinha (P. K. Bose, with him) for the appel lant.
N.C. Chatterjee and Panchanan Ghosh (Chandra Narayan Naik, with them) for the respondent. 1950.
December 1.
The Judgment of the Court was deliv ered by PATANJALI SASTRI J.
This appeal arises out of a suit brought by the respondent in the court of the Subordinate Judge, Dhanbad, for recovery of arrears of royalty and cess from the appellant and another alleged to be due under a compromise decree passed on the 6th March, 1923, in a previ ous suit between the predecessors in interest of the par ties.
The only plea which is material for the purpose of this appeal is that the compromise decree not having been registered 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated Summary: iminal Appeal No. 40 of 1951, 127 Appeal from the Judgment and Order dated the 1st June, 1951, of the High Court of Judicature in Assam (Thadani C.J. and Ram Labhaya J.,) in Criminal Reference No. I of 1951, arising out of Judgment and Order dated the 15th November, 1950, of the Court of the Additional District Magistrate, Lakhimpur, in Case No. 1126C of 1950.
Jindra Lal for the appellant.
Nuruddin Ahmed for the respondent.
October 23.
The Judgment of the Court was delivered by CHANDRASEKHARA AIYAR J.
Rameshwar Bhartia, the appellant, is a shopkeeper in Assam.
He was prosecuted for storing paddy without a licence in excess of the quantity permitted by the Assam Food Grains Control Order, 1947.
He admitted storage and possession of 550 maunds of paddy, but pleaded that he did not know that any licence was necessary.
The 'Additional District Magistrate recorded a plea of guilty, but imposed him a fine of Rs. 50 only, as he considered his ignorance of the provisions of 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated Summary: Appeal No. 388 of 1960.
Appeal by special leave from the judgment and order dated February 3, 1959, of the Patna High Court in Election Appeal No. 10 of 1958.
section P. Varma, for the appellant.
L. K. Jha and D. Govardhan, for respondent No. 1.
L. K. Jha and K. K. Sinha, for respondent No. 2. 1960.
November 17.
The Judgment of the Court was delivered by 471 GAJENDRAGADKAR, J.
Is the appellant Ram Padarath Mahto disqualified for membership of the Bihar Legislature under section 7(d) of the Representation of the People Act, 1951 (hereafter called the Act)? That is the short question which arises for our decision in the present appeal by special leave.
The appellant was one of the candidates for the Dalsinghsarai Constituency in the District of Darbhanga in Bihar for the State Legislature.
The said Constituency is a Double Member Constituency; it was required to elect two members, one for the general and the other for the reserved seat for scheduled castes in the Bihar

In [38]:
gsummaries=["Appeal No. LXVI of 1949. Appeal from the High Court of judicature, Bombay, in a reference under section 66 of the Indian Income tax Act, 1022. K.M. Munshi (N. P. Nathvani, with him), for the appel lant. ' M.C. Setalvad, Attorney General for India (H. J. Umrigar, with him), for the respondent. 1950. May 26. The judgment of the Court was delivered by MEHR CHAND MAHAJAN J. This is an appeal against a judgment of the High Court of Judicature at Bombay in an income tax matter and it raises the question whether munici pal property tax and urban immoveable property tax payable under the relevant Bombay Acts are allowable deductions under section 9 (1) (iv) of the Indian Income tax Act. The assessee company is an investment company deriving its income from properties in the city of Bombay. For the assessment year 1940 41 the net income of the assessee under the head property was computed by the Income tax Officer in the sum of Rs. 6,21,764 after deducting from gross rents certain payments. The company had paid during the relevant year Rs. 1,22,675 as municipal property tax and Rs. 32,760 as urban property tax. Deduction of these two sums was claimed under the provisions of section 9 the Act. Out of the first item a deduction in the sum of Rs. 48,572 was allowed on the ground that this item represented tenants ' burdens paid by the assessee, otherwise the claim was disal lowed. The, appeals of the assessee to the Appellate As sistant Commissioner and to the Income tax Appellate Tribu nal were unsuccessful. The Tribunal, however, agreed to refer two questions of law to the High Court of Judicature at Bombay, namely, (1) Whether the municipal taxes paid by the applicant company are an allowable deduction under 555 the provisions of section 9 (1) (iv) of the Indian Income tax Act; (2) Whether the urban immoveable property taxes paid by the applicant company are an allowable deduction under section 9 (1) (iv) or under section 9 (1) (v) of the Indian Income tax Act. The High Court of Judicature at Bombay, in its judgment dated 14th December, 1949, held that the urban immoveable property tax paid by the assessee company is not an allowable deduction under section 9 (1) (iv) of the Indian Income tax Act. The assessee company has appealed against the judgment of the High Court of Judicature at Bombay. The assessee company has also appealed against the judgment of the High Court of Judicature at Bombay in the reference made under section 66 of the Indian Income tax Act.", "Civil Appeal No.94 of 1949. 107 834 Appeal from a judgment and decree of the High Court of Judi cature at Patna in Appeal from Appellate Decree No. 97 of 1946 (Mannohar Lall and Mukherji JJ.) dated 23rd Decem ber, 1947, confirming the judgment of the District Judge of Purulia in Appeal No. 159 of 1944. S.P. Sinha (P. K. Bose, with him) for the appel lant. N.C. Chatterjee and Panchanan Ghosh (Chandra Narayan Naik, with them) for the respondent. 1950. December 1. The Judgment of the Court was deliv ered by PATANJALI SASTRI J. This appeal arises out of a suit brought by the respondent in the court of the Subordinate Judge, Dhanbad, for recovery of arrears of royalty and cess from the appellant and another alleged to be due under a compromise decree passed on the 6th March, 1923, in a previ ous suit between the predecessors in interest of the par ties. The only plea which is material for the purpose of this appeal is that the compromise decree not having been registered was inadmissible in evidence. The courts below held that the document did not require registration and gave effect to its terms in decreeing the suit. The second defendant has preferred this appeal. The facts are not now in dispute and may be briefly stated. On 11th March, 1921, one Kumar Krishna Prasad Singh (hereinafter referred to as Kumar) granted a perma nent lease of the right to the underground coal in 5,800 bighas of land belonging to him to Shibsaran Singh and Sitaram Singh (hereinafter referred to as the Singhs) by a registered patta stipulating for a salami of Rs. 8,000 and royalty at the rate of 2a. per ton of coal raised subject to a minimum of Rs. 8,000 and for certain other cesses and interest. On 7th June, 1921, Kumar executed another perma nent patta leasing the right to the coal in 500 bighas out of the 5,800 bighas of land to the Singhs. On 11th March, 1921, the Singhs executed a patta leasing the right to the coal in 5,800 bighas of land to the appellant and the second defendant.", "iminal Appeal No. 40 of 1951, 127 Appeal from the Judgment and Order dated the 1st June, 1951, of the High Court of Judicature in Assam (Thadani C.J. and Ram Labhaya J.,) in Criminal Reference No. I of 1951, arising out of Judgment and Order dated the 15th November, 1950, of the Court of the Additional District Magistrate, Lakhimpur, in Case No. 1126C of 1950. Jindra Lal for the appellant. Nuruddin Ahmed for the respondent. October 23. The Judgment of the Court was delivered by CHANDRASEKHARA AIYAR J. Rameshwar Bhartia, the appellant, is a shopkeeper in Assam. He was prosecuted for storing paddy without a licence in excess of the quantity permitted by the Assam Food Grains Control Order, 1947. He admitted storage and possession of 550 maunds of paddy, but pleaded that he did not know that any licence was necessary. The 'Additional District Magistrate recorded a plea of guilty, but imposed him a fine of Rs. 50 only, as he considered his ignorance of the provisions of the Food Grains Control Order to be genuine. The stock of paddy was left in the possession of the appellant by the Procurement Inspector under a Jimmanama or security bond executed in his favour. He was subsequently unable to produce it before the court, as the whole of it was taken away by a Congress M.L.A. for affording relief to those who suffered in the earthquake, and so, the appellant was ordered to procure a similar quantity of paddy after taking an appropriate licence, and to make over the same to the procurement department payment of the price. The District Magistrate, being moved to do so by the procurement department, referred the case to the High Court under section 438, Criminal Procedure Code, for enhancement of the sentence, as in his opinion the sentence was unduly lenient and the Jimmanama, which was admittedly broken, should have been forfeited. 128 The reference was accepted by the High Court, and the sent ence was enhanced to rigorous ' imprisonment for six months and a fine of Rs. 1,000. As regards the Jimmanama, the High Court held that it was forfeited, and the appellant was sentenced to rigorous imprisonment for six months and a fine of Rs. 1,000. The appellant appealed to the Supreme Court, and the appeal was allowed, and the sentence was reduced to imprisonment for six months and a fine of Rs. 500. The appellant has appealed to this Court. The appellant contends that the High Court erred in holding that the Jimmanama was forfeited, and that the sentence should have been enhanced to rigorous imprisonment for six months and a fine of Rs. 1,000.", "Appeal No. 388 of 1960. Appeal by special leave from the judgment and order dated February 3, 1959, of the Patna High Court in Election Appeal No. 10 of 1958. section P. Varma, for the appellant. L. K. Jha and D. Govardhan, for respondent No. 1. L. K. Jha and K. K. Sinha, for respondent No. 2. 1960. November 17. The Judgment of the Court was delivered by 471 GAJENDRAGADKAR, J. Is the appellant Ram Padarath Mahto disqualified for membership of the Bihar Legislature under section 7(d) of the Representation of the People Act, 1951 (hereafter called the Act)? That is the short question which arises for our decision in the present appeal by special leave. The appellant was one of the candidates for the Dalsinghsarai Constituency in the District of Darbhanga in Bihar for the State Legislature. The said Constituency is a Double Member Constituency; it was required to elect two members, one for the general and the other for the reserved seat for scheduled castes in the Bihar Legislative Assembly. It appears that the said Constituency called upon voters to elect members on January 19, 1957. January 29, 1957 was fixed as the last date for the filing of the nomination papers. The appellant filed his nomination paper on January 28, 1957, and on the next day seven other members filed their nomination papers. On February 1, 1957, the nomination paper filed by the appellant was rejected by the returning officer on two grounds; he held that the appellant being an Inspector of Co operative Societies was a Government servant at the material time and so was disqualified from standing for election. He also found that the appellant was a member of a joint and undivided Hindu family which carried on the business of Government as stockiest of grain under a contract between the Government of Bihar and a firm of the joint family known as Nebi Mahton Bishundayal Mahto. Thereafter the election was duly held, and Mr. Mishri Singh and Mr. Baleshwar Ram, respondents 1 and 2 were declared duly elected to the general and reserved seats respectively. The appellant filed an election petition in the Patna High Court, and the said petition was dismissed by the High Court on February 3, 1959. The appellant has appealed to this Court. The appellant contends that he was not disqualified from standing for election on the ground that he was a Government servant at the material time, because he was not a Government servant at the material time, i.e., on January 19, 1957, the date of the election. The appellant further contends that he was not disqualified from standing for election on the ground that he was a member of a joint and undivided Hindu family which carried on the business of Government as stockiest of grain under a contract between the Government of Bihar and a firm of the joint family known as Nebi Mahton Bishundayal Mahto, because he was not a member of such a joint and undivided Hindu family at the material time, i.e., on January 19, 1957, the date of the election.", "Appeal No. 198 of 1954. Appeal from the judgment and order dated October 16, 1952, of the former Nagpur High Court in Misc. Petn.; No. 1231 of 1951. M. section K. Sastri, for the appellant. H. L. Khaskalam, B. K. B. Naidu and I. N. Shroff, for the respondent. 64 502 1960. November 18. The Judgment of the Court was delivered by IMAM, J. This is an appeal from the judgment of the Nagpur High Court dismissing the appellants petition under articles 226 and 227 of the Constitution of India. The High Court certified under article 132(1) of the Constitution that the case involved a substantial question of law as to the interpretation of the Constitution. Hence the present appeal. The appellant was the Ruler of the State of Baster. After the passing of the Indian Independence Act, 1947, the appellant executed an Instrument of Accession to the Dominion of India on August 14, 1947. Thereafter, he entered into an agreement with the Dominion of India popularly known as The Stand Still Agreement. On December 15, 1947, he entered into an agreement with the Government of India whereby he ceded the State of Baster to the Government of India to be integrated with the Central Provinces and Berar (now the State of Madhya Pradesh) in such manner as the Government of India thought fit. Con sequently the Governments in India came to have exclusive and plenary authority, jurisdiction and powers over the Baster State with effect from January 1, 1948. The Legislature of the State of Madhya Pradesh passed the Madhya Pradesh Abolition of Proprietary Rights (Estates, Mahals, Alienated Lands) Act, 1950 (Madhya Pradesh Act 1 of 1951), hereinafter referred to as the Act, which received the assent of the President of India on January 22, 1951. The preamble of the Act stated that it was one to provide for the acquisition of the rights of proprietors in estates, mahals, alienated villages and alienated lands in Madhya Pradesh and to provide for the abolition of such rights. The Act was enacted by the Legislature of Madhya Pradesh. The Act was subsequently assented to by the President of India on February 15, 1951."]

In [50]:
import itertools
from rouge_score import rouge_scorer

# Assuming summary_content and gsummaries are defined elsewhere in your code.
# Example placeholders:
# summary_content = {"file1.txt": "Summary 1", "file2.txt": "Summary 2", ...}
# gsummaries = ["Generated Summary 1", "Generated Summary 2", ...]

def rouge_score(summary_content, gsummaries):
    """
    Calculates the average ROUGE-L score between reference and generated summaries.

    Args:
        summary_content (dict): A dictionary of reference summaries (filename: content).
        gsummaries (list): A list of generated summaries.

    Returns:
        float: The average ROUGE-L score.
    """
    try:
        scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
        scores = []
        for i, (filename, content) in enumerate(itertools.islice(summary_content.items(), 5)):
            if i < len(gsummaries):
                score = scorer.score(content, gsummaries[i])
                print(f"Actual Summary (File: {filename}):\n{content}\n")
                print(f"Generated Summary:\n{gsummaries[i]}\n")
                scores.append(score['rougeL'].fmeasure)
            else:
                break  # Stop if gsummaries are exhausted
        if scores:
            return sum(scores) / len(scores)
        else:
            return 0.0  # Return 0 if no scores were calculated
    except ImportError:
        print("Please install the rouge_score library: pip install rouge_score")
        return None

def rouge_n_score(summary_content, gsummaries, n=1):
    """
    Calculates the average ROUGE-N score between reference and generated summaries.

    Args:
        summary_content (dict): A dictionary of reference summaries (filename: content).
        gsummaries (list): A list of generated summaries.
        n (int): The n-gram order.

    Returns:
        float: The average ROUGE-N score.
    """
    try:
        scorer = rouge_scorer.RougeScorer([f'rouge{n}'], use_stemmer=True)
        scores = []
        for i, (filename, content) in enumerate(itertools.islice(summary_content.items(), 5)):
            if i < len(gsummaries):
                score = scorer.score(content, gsummaries[i])
                scores.append(score[f'rouge{n}'].fmeasure)
            else:
                break
        if scores:
            return sum(scores) / len(scores)
        else:
            return 0.0
    except ImportError:
        print("Please install the rouge_score library: pip install rouge_score")
        return None

def rouge_2_score(summary_content, gsummaries):
    return rouge_n_score(summary_content, gsummaries, 2)

def rouge_1_score(summary_content, gsummaries):
    return rouge_n_score(summary_content, gsummaries, 1)

# Example Usage (replace with your actual data):
# Assuming summary_content and gsummaries are populated with your data.
# Example:
# summary_content = {"file1.txt": "Reference summary 1", "file2.txt": "Reference summary 2", ...}
# gsummaries = ["Generated summary 1", "Generated summary 2", ...]

# Ensure summary_content and gsummaries are defined.

if 'summary_content' in locals() and 'gsummaries' in locals():

  rouge_l = rouge_score(summary_content, gsummaries)
  rouge_1 = rouge_1_score(summary_content, gsummaries)
  rouge_2 = rouge_2_score(summary_content, gsummaries)

  if rouge_l is not None:
      print(f"ROUGE-L Score: {rouge_l}")
  if rouge_1 is not None:
      print(f"ROUGE-1 Score: {rouge_1}")
  if rouge_2 is not None:
      print(f"ROUGE-2 Score: {rouge_2}")

else:
    print("summary_content or gsummaries not defined. Please ensure your data is loaded.")

Actual Summary (File: 1.txt):
The charge created in respect of municipal property tax by section 212 of the City of Bombay Municipal Act, 1888, is an "annual charge not being a capital charge" within the mean ing of section 9 (1) (iv) of the Indian Income tax Act, 199.2, and the amount of such charge should therefore be deducted in computing the income from such property for the purposes of section 9 of the Indian Income tax Act.
The charge in respect of urban immoveable property tax created by the Bombay Finance Act, 1939 is similar in character and the amount of such charge should also be deducted.
The expression "capital charge" in s.9(1) (iv) means a charge created for a capital sum,that is to say, a charge created to. ' secure the discharge of a liability of a capi tal nature; and an "annual charge" means a charge to secure an annual liabili ty. 554


Generated Summary:
Appeal No. LXVI of 1949. Appeal from the High Court of judicature, Bombay, in a reference under section 66 of th

In [ ]:
def save_preprocessed_text(files_content, output_directory):
    os.makedirs(output_directory, exist_ok=True) 
    for filename, content in files_content.items():
        preprocessed_text = preprocess_text(content)
        output_filename = os.path.join(output_directory, filename)
        with open(output_filename, 'w', encoding='utf-8') as outfile:
            outfile.write(' '.join(preprocessed_text)) 

In [ ]:
output_directory = "D://Legal//IND//IN-Abs//train-data//judgement_pre"
save_preprocessed_text(files_content, output_directory)